In [10]:
from polygon import RESTClient
import sqlite3
# from tqdm import tqdm
import pickle

from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import pandas as pd

from tqdm.notebook import tqdm

# time.sleep(60)


###### get list of SP500 symbols ######
# data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# table = data[0]
# stock_list = table['Symbol'].values.tolist()

with open("../Analysis/screener/optionable_ticker_for_val", "rb") as f:
    stock_list = pickle.load(f)
print(len(stock_list))


# test
# stock_list = ['AAPL']
# stock_list = stock_list[:3]

1781


In [2]:
# create_table_sql = '''
# create table if not exists stock_price_minute_optionable(
#     id integer primary key,
#     symbol text not null,
#     datetime not null,
#     open not null,
#     high not null,
#     low not null,
#     close not null,
#     volume not null,
#     trxn integer
#     -- foreign key (stock_id) references stock (id)
# )
# '''
# def create_table(cursor, create_table_sql):
#     """ create a table from the create_table_sql statement
#     :param conn: Connection object
#     :param create_table_sql: a CREATE TABLE statement
#     :return:
#     """
#     try:
#         cursor.execute(create_table_sql)
#     except Error as e:
#         print(e)

In [ ]:
# time.sleep(60)

now = datetime.now()
two_yrs_ago = now - relativedelta(years=2)
glb_start = two_yrs_ago
glb_end = now.date() - relativedelta(days=1)
print(glb_start, glb_end)


already_data_in_database = False
if already_data_in_database:
    connection = sqlite3.connect("E:\databases\yolo.db")
    cursor = connection.cursor()
    cursor.execute("select symbol, max(id) from stock_price_minute group by symbol")
    cur = cursor.fetchall()
    symbol_already_in_database = [x[0] for x in cur]
    max_id = max([x[1] for x in cur])
    i = max_id + 1
else:
    i = 1
    symbol_already_in_database = []

client = RESTClient("CtCiOlTjiJGq_5NLZNQbMM6Y0GExYsJ5") # api_key is used

api_call_num = 1
for ticker in tqdm(stock_list):
    start = glb_start
    end = glb_end
    
    if ticker not in symbol_already_in_database:

        if api_call_num % 5 == 0:
#             print(f'{datetime.now()}... reached api call limit, waiting for 60 seconds...')
            time.sleep(60)
        aggs = client.get_aggs(ticker, 1, "minute", start, end, limit=50000)
        api_call_num += 1

        all_lst = []
        while 1:
            tmp_lst = []
            for bar in aggs:
                row = [bar.timestamp,
                        datetime.fromtimestamp(bar.timestamp / 1000),
                        bar.open, bar.high, bar.low, bar.close,
                        bar.volume, bar.vwap, bar.transactions, bar.otc]
                tmp_lst.append(row)
            df_tmp = pd.DataFrame(tmp_lst, columns=['timestamp', 'time', 'open', 'high', 'low','close', 'volume', 'vwap', 'transactions', 'otc'])
            all_lst.append(df_tmp)
            # print(df_tmp.shape)
            start = df_tmp['timestamp'].values[-1]
#             print(start)

            # print(f'start date: {datetime.fromtimestamp(start / 1000).date()}, end date: {end}')
            if (datetime.fromtimestamp(start / 1000).date() >= end - relativedelta(days=3)): # -3days for corner case where run date is on a holiday Monday, the latest data we have is on the previous Friday
                break

            if api_call_num % 5 == 0:
                # print(f'{datetime.now()}... reached api call limit, waiting for 60 seconds...')
                time.sleep(60)
            aggs = client.get_aggs(ticker, 1, "minute", start, end, limit=50000)
            api_call_num += 1
            
        df = pd.concat(all_lst, ignore_index=True).drop_duplicates()
        # df.index = df.time
        # df = df.resample('1min').ffill()
        # df = df[(df.time.astype(str).str[11:]>='04:00:00') & (df.time.astype(str).str[11:]<='19:59:00')]

        connection = sqlite3.connect("E:\databases\yolo.db")
        connection.row_factory = sqlite3.Row
        cursor = connection.cursor()
        
#         create_table(cursor, create_table_sql)
        
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO stock_price_minute_optionable (id, symbol, datetime, open, high, low, close, volume, trxn)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (i, ticker, row['time'].tz_localize(None).isoformat(), row['open'], row['high'], row['low'], row['close'], row['volume'], row['transactions']))
            i += 1
    
        connection.commit()
        print(f'finished {ticker}', '\n')

2021-02-19 01:02:34.907793 2023-02-18


  0%|          | 0/1781 [00:00<?, ?it/s]

finished A 

finished AA 

finished AAL 

finished AAON 

finished AAP 

finished AAPL 

finished AAWW 

finished AB 

finished ABB 

finished ABBV 

finished ABC 

finished ABCB 

finished ABCL 

finished ABCM 

finished ABG 

finished ABM 

finished ABNB 

finished ABR 

finished ABT 

finished ACA 

finished ACAD 

finished ACDC 

finished ACGL 



### quota calculation

num of bars: 60 bars/hour * 14 hours/day * 250 days/year * 2 year = 420000 bars

420000 / (50000 * 5) = 1.68min

In [8]:
connection.commit()